In [1]:
import sys
import os

import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
# retrive data 
df = pd.read_csv("./core_exchange.csv", 
                 index_col=0, 
                 parse_dates=True)

df.head()

/tmp/ipykernel_386851/208368346.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv("./core_exchange.csv",


,currency_name,bank_name,buy_rate,sell_rate,increment_amount,created_at,updated_at
id,,,,,,,
1300,USD,Hibret Bank,105.1586,118.8292,NaN,2024-08-25 09:05:28.687571+00,2024-08-25 09:05:28.687592+00
2,USD,Bank of Abyssinia,104.6247,117.1797,NaN,2024-08-23 03:05:19.612498+00,2024-08-23 03:05:19.612571+00
3,GBP,Bank of Abyssinia,130.4970,146.1566,NaN,2024-08-23 03:05:19.618773+00,2024-08-23 03:05:19.618785+00
4,EUR,Bank of Abyssinia,116.6042,130.5967,NaN,2024-08-23 03:05:19.619882+00,2024-08-23 03:05:19.619893+00
5,AED,Bank of Abyssinia,28.4879,31.9064,NaN,2024-08-23 03:05:19.620869+00,2024-08-23 03:05:19.620877+00


In [ ]:
print("Sum of total rows in the DataFrame:")
print(len(df))
print("Missing values in each column:")
print(df.isnull().sum())

print("\nData types of each column:")
print(df.dtypes)

print("\nSummary statistics of the DataFrame:")
print(df.describe())

Missing values in each column:
currency_name            0
bank_name                0
buy_rate                 0
sell_rate                0
increment_amount    134801
created_at               0
updated_at               0
dtype: int64

Data types of each column:
currency_name        object
bank_name            object
buy_rate            float64
sell_rate           float64
increment_amount    float64
created_at           object
updated_at           object
dtype: object

Summary statistics of the DataFrame:
            buy_rate      sell_rate  increment_amount
count  134801.000000  134801.000000               0.0
mean       90.706025     179.986443               NaN
std        63.469969    7270.592196               NaN
min        -1.000000      -1.000000               NaN
25%        32.236600      33.387700               NaN
50%       108.551000     118.829200               NaN
75%       135.129300     138.657500               NaN
max      1142.526500  883430.000000               NaN
